In [1]:
import numpy as np
import fmm_tree
import tree_build_routines as tbr
import list_build_routines as lbr
import numpy.matlib
import itertools
%load_ext line_profiler

In [4]:
npts = 50

x = np.linspace(1.,2.,npts+1) 
xpos, zpos = np.meshgrid(x, x)
gvals = np.ones((npts+1)**2) 
pval = 5
mx = 10.
nvorts = xpos.size
ccnt, rcnt = 2, 2

mytree = tbr.make_tree(xpos.flatten(), zpos.flatten(), gvals, pval, mx, nvorts, ccnt, rcnt)
tbr.build_tree(mytree)
%timeit -t lbr.build_tree_lists(mytree)
%lprun -f lbr.build_node_lists lbr.build_tree_lists(mytree)

10 loops, best of 3: 149 ms per loop
